In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
devices = tf.config.list_physical_devices('CPU')
cpu_name = devices[0].name

devices = tf.config.list_physical_devices('CPU')
print(devices[0])
# You'll see that the name will look something like "/physical_device:CPU:0"
# Just take the CPU:0 part and use that as the name
cpu_name = "CPU:0"

In [ ]:
pixels = 224
MODULE_HANDLE = 'https://tfhub.dev/tensorflow/resnet_50/feature_vector/1'
IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {}".format(MODULE_HANDLE, IMAGE_SIZE))

In [ ]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']
#data, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples), info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes


In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [ ]:
BATCH_SIZE = 32
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [ ]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

In [ ]:
do_fine_tuning = False #@param {type:"boolean"}

In [ ]:
def build_and_compile_model():
  print("Building model with", MODULE_HANDLE)
  feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE + (3,), 
                                   trainable=do_fine_tuning)
  model = tf.keras.Sequential([
      feature_extractor,
      tf.keras.layers.Dense(num_classes, activation='softmax')
  ])
  model.summary()

  optimizer = tf.keras.optimizers.SGD(lr=0.002, momentum=0.9) if do_fine_tuning else 'adam'
  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model

In [ ]:
one_strategy = tf.distribute.OneDeviceStrategy(device=cpu_name)
with one_strategy.scope():
  model = build_and_compile_model()

In [ ]:
EPOCHS = 5
hist = model.fit(train_batches,
                 epochs=EPOCHS,
                 validation_data=validation_batches)